#### Formating cases.last_update: removing T

In [1]:
%run cases.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1936: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:956: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
